In [1]:
from pyspark import SparkContext
from pyspark import AccumulatorParam
sc = SparkContext("local", "P4")

In [2]:
source = sc.textFile("source.csv")
source.take(1)

[u'"FROST, CARMILLA","AA2 35"']

In [3]:
def comic_KV(x):
    K_V = x.split('","')
    return (K_V[1][0:-1], [K_V[0][1:]])
def node_KV(x):
    K_V = x.split('","')
    return (K_V[1][0:-1], K_V[0][1:])
def get_neighbors(val):
    c_n = val[1]
    neighbors = c_n[1][:]
    neighbors.remove(c_n[0])
    return (c_n[0], neighbors)
def group_neighbors(x, y):
    return set(x) | set(y)

nodes = source.map(node_KV)
comics = source.map(comic_KV)
comics = comics.reduceByKey(lambda x,y: x + y)

In [4]:
nodes_neighbors = nodes.join(comics).map(get_neighbors).reduceByKey(group_neighbors)
sorted_neighbors = nodes_neighbors.sortByKey()

In [257]:
class DistanceAccumulatorParam(AccumulatorParam):
    def zero(self, initialValue):
        return initialValue

    def addInPlace(self, v1, v2):
        for key in v2.keys():
            if key not in v1:
                v1[key] = v2[key]
        return v1
    
def ss_bfs_accum(rdd, root, diameter = -1):
    distance_hash = rdd.context.accumulator({root: 0}, DistanceAccumulatorParam())
    next_hop = rdd.lookup(root)[0]
    hops = 1
    while (hops <= diameter or diameter < 0) and len(next_hop) > 0:
        next_rdd = rdd.filter(lambda x: x[0] in next_hop)
        next_rdd.foreach(lambda x: distance_hash.add({x[0] : hops}))
        next_hop = set(next_rdd.flatMap(lambda x: x[1]).collect()) - set(distance_hash.value.keys())
        hops += 1
    return distance_hash.value

def update_accum(accumulator, node, hops):
    if node[0] in neighbors and node[1] == 0:
        accumulator.add(1)
        return (node[0], hops)
    else:
        return node

def ss_bfs(rdd, root, diameter = -1):
    next_hop = rdd.lookup(root)[0]
    distances = rdd.map(lambda x: (x[0], 0))
    accum = rdd.context.accumulator(0)
    hops = 1
    while (hops <= diameter or diameter < 0) and len(next_hop) > 0:
        reached = accum.value
        accum.value = 0
        distances = distances.map(lambda x: update_accum(accum, x, next_hop, hops))
        next_hop = set(rdd.filter(lambda x: x[0] in next_hop).flatMap(lambda x: x[1]).collect()) - set([root])
        if reached == accum.value:
            return distances
        hops += 1
    return distances

In [245]:
roots = [u'CAPTAIN AMERICA', u'MISS THING/MARY', u'ORWELL']
cap_dict = ss_bfs_accum(sorted_neighbors, roots[0],1)
cap_set = set(cap_dict.keys())
    

In [77]:
%%timeit
for r in roots:
    print len(ss_bfs(sorted_neighbors, r))

TypeError: object of type 'int' has no len()

In [263]:
ss_bfs(sorted_neighbors, u'CAPTAIN AMERICA', 1).filter(lambda x: )

[(u'24-HOUR MAN/EMMANUEL', 0),
 (u'3-D MAN/CHARLES CHAN', 1),
 (u'4-D MAN/MERCURIO', 1),
 (u'8-BALL/', 1),
 (u'A', 1),
 (u"A'YIN", 1),
 (u'ABBOTT, JACK', 1),
 (u'ABCISSA', 1),
 (u'ABEL', 1),
 (u'ABOMINATION | MUTANT', 1)]

In [251]:
#cap_set - accum_set
len(accum_set)

6369